#### Loads dev.in as a list, and separates each sentence

In [9]:
def load_original_train():
    with open("train", encoding="utf-8") as file:
        train_list = file.readlines()
        train_list = [x.strip() for x in train_list]
        return train_list
    
train_list = load_original_train()

In [10]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}

for i in range(len(train_list)):
    annotated_word = train_list[i]
    each_word = annotated_word.split(" ")
    if (len(each_word) == 2):
        if each_word[1] == "O":
            annotationDict["O"] += 1
        elif each_word[1] == "I-positive":
            annotationDict["I-positive"] += 1
        elif each_word[1] == "B-positive":
            annotationDict["B-positive"] += 1
        elif each_word[1] == "I-negative":
            annotationDict["I-negative"] += 1
        elif each_word[1] == "B-negative":
            annotationDict["B-negative"] += 1
        elif each_word[1] == "I-neutral":
            annotationDict["I-neutral"] += 1
        elif each_word[1] == "B-neutral":
            annotationDict["B-neutral"] += 1

print(annotationDict)

{'O': 24242, 'I-positive': 607, 'B-positive': 1208, 'I-negative': 133, 'B-negative': 382, 'I-neutral': 23, 'B-neutral': 65}


In [12]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in dev_in_list]
        return dev_in_list
    
def load_modified_train_file():
    with open("modified_train", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        return modified_train_list
        
def split_into_sentences(dev_in_list):
    sentences = []
    each_sentence = []
    for word in dev_in_list:
        if word == "":
            sentences.append(each_sentence)
            each_sentence = []
        else:
            each_sentence.append(word)
        
    return sentences

modified_train_list = load_modified_train_file()   
dev_in_list = load_files()
sentences = split_into_sentences(dev_in_list)

#### Giant Emission and Giant Transition Dict

In [17]:
def count_word_tag_pair_dict():
    word_tag_pair_dict= {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair_dict.get(word_tag_pair) == None:
            word_tag_pair_dict[word_tag_pair] = 1
        else:
            word_tag_pair_dict[word_tag_pair] += 1
    return word_tag_pair_dict

def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    

def store_estimate_emission_fix():
    giantEmissionDict = {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair != "":
            word_tag_pair_list = word_tag_pair.split(" ")
            countTop = wordTagPairDict[word_tag_pair]
            countBottom = annotationDict[word_tag_pair_list[1]]
            emission = countTop/countBottom
            if giantEmissionDict.get(word_tag_pair) == None:
                giantEmissionDict[word_tag_pair] = countTop/countBottom
    return giantEmissionDict

def store_estimate_transition():
    giantTransitionDict = {}
    finalGiantTransitionDict = {}
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if giantTransitionDict.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
    for eachTransition in giantTransitionDict:
        eachTransitionList = eachTransition.split(" ")
        startState = eachTransitionList[0]
        countTop = giantTransitionDict[eachTransition]
        countBottom = totalStateNumber[startState]
        transitionParam = countTop/countBottom
        finalGiantTransitionDict[eachTransition] = transitionParam
        
    return finalGiantTransitionDict

def separate_tweets():
    countStart = 1
    # counting starts
    giant_each_tweet = []
    each_tweet = []
    totalStateNumber = {}
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giant_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[1])
    for i in range(len(giant_each_tweet)):
        giant_each_tweet[i].insert(0, 'START')
        giant_each_tweet[i].append('STOP')
    
    for each_tweet in giant_each_tweet:
        for tag in each_tweet:
            if totalStateNumber.get(tag) == None:
                totalStateNumber[tag] = 1
            else:
                totalStateNumber[tag] += 1

    return giant_each_tweet, countStart, totalStateNumber

modifiedWordDict = get_word_dict()
wordTagPairDict = count_word_tag_pair_dict()
giantEmissionDict = store_estimate_emission_fix()
giantTaggedEachTweet, countStart, totalStateNumber = separate_tweets()
print(giantEmissionDict["are O"])
giantTransitionDict = store_estimate_transition()
# print(giantTransitionDict)
# print(len(giantTransitionDict))

0.0054038445672799276


In [14]:
def forward_prob(i,u, sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    last_word = sentence[len(sentence)-1]
    # base case
    if i == 1:
        if giantTransitionDict.get("START" + " " + u) != None:
            a = giantTransitionDict["START" + " " + u]#transition from start to u
            return a
        else:
#             print(u)
            return 0
    # recursive case
    else:
        word_at_u = sentence[i]
        if word_at_u in modifiedWordDict:
            word_at_u = word_at_u
        else:
            word_at_u = "#UNK#"
        forward_prob_list = []
        for v in all_tags:
            if giantTransitionDict.get(v + " "+ u) != None and giantEmissionDict.get(word_at_u+" "+v ) != None:
                a = giantTransitionDict[v +" "+ u]#transition from v to u #read from dict
                b = giantEmissionDict[word_at_u + " "+ v]#emission from v to xi #read from dict
                forward_prob_from_v = forward_prob(i-1, v, sentence)*a*b
                forward_prob_list.append(forward_prob_from_v)
        return sum(forward_prob_list)
    
def back_prob(i,u, sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    last_word = sentence[len(sentence)-1]
    n = len(sentence)
    # base case
    if i == n:
        if giantTransitionDict.get(u + " " + "STOP") != None and giantEmissionDict.get(last_word + " " + u) != None:
            a = giantTransitionDict[u + " " + "STOP"]#transition from u to stop
            b = giantEmissionDict[last_word + " " + u]#emission from u to xn
            return a*b
        else:
            return 0
    # recursive case
    else:
        word_at_u = sentence[i]
        if word_at_u in modifiedWordDict:
            word_at_u = word_at_u
        else:
            word_at_u = "#UNK#"
        back_prob_list = []
        for v in all_tags:
            if giantTransitionDict.get(u + " "+ v) != None and giantEmissionDict.get(word_at_u + " " + u) != None:
                a = giantTransitionDict[u + " " + v]#transition from u to v
                b = giantEmissionDict[word_at_u + " " + u]#emission from u to xi
                back_prob_from_v = back_prob(i + 1, v, sentence) * a * b
                back_prob_list.append(back_prob_from_v)
#                 print(back_prob_list)

        return sum(back_prob_list)


In [18]:
def combine_path(best_path, sentence):
#     print(best_path)
#     print(sentence)
#     print(len(best_path))
#     print(len(sentence))
    result = ""
    for i in range(len(sentence)):
        if (sentence[i] == ''):
            result += " \n"
        else:
            result += sentence[i] + " " + best_path[i] + "\n"

    return result


In [19]:
def max_marginal(sentence):
    sentence.append("")
    #sentence is a list containing the words
    # this is only for one sentence
    opti_path = []
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    for i in range(len(sentence)):
        opti_y_dict = {}
        for tag in all_tags:
            opti_y_at_i = forward_prob(i,tag, sentence)*back_prob(i,tag, sentence)
            opti_y_dict[tag] = opti_y_at_i
        
        max_i_path = max(opti_y_dict, key=opti_y_dict.get)
        opti_path.append(max_i_path)
    return opti_path

# dev_in_list is a giant list
# do a split according to ""
# giantStringToBeWritten = ""
# for sentence in sentences:
#     singleStringToBeWritten = max_marginal(sentence)
#     giantStringToBeWritten += singleStringToBeWritten
giantStringToBeWritten = ""
for i in range(len(sentences)):
#     if i ==1:
    best_path = max_marginal(sentences[i])
    singleStringToBeWritten = combine_path(best_path, sentences[i])
    print(singleStringToBeWritten)
    giantStringToBeWritten += singleStringToBeWritten
print(giantStringToBeWritten)

When O
I O
called O
this O
morning O
, O
I O
didn't O
think O
I O
would O
be O
able O
to O
get O
in O
at O
12 O
, O
but O
I O
was O
able O
to O
get O
in O
, O
along O
with O
four O
other O
guests O
. O
 
 

The O
tuna O
and O
wasabe O
potatoes O
are O
excellent O
. O
 
 

My O
favortie O
pizza O
joint O
in O
Seattle O
 
 

Food O
wise O
, O
its O
ok O
but O
a O
bit O
pricey O
for O
what O
you O
get O
considering O
the O
restaurant O
isn't O
a O
fancy O
place O
. O
 
 

– O
tucked O
away O
over O
by O
the O
Beverly O
Center O
. O
 
 

My O
mouth O
felt O
very O
dry O
afterwards O
and O
I O
had O
that O
signature O
? O
MSG O
? O
taste O
lingering O
in O
my O
throat O
after O
I O
left O
the O
restaurant O
. O
 
 



KeyboardInterrupt: 